In [1]:
import pykeen

In [2]:
import os, os.path as path

In [3]:
from pykeen.pipeline import pipeline

D:\anaconda3\lib\site-packages\pykeen\evaluation\evaluator.py:16: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
from pykeen.triples import TriplesFactory

In [10]:
from pykeen.models.predict import get_tail_prediction_df

In [73]:
tf = TriplesFactory.from_path('defillama_kg.tsv', create_inverse_triples=True)

In [87]:
tf = TriplesFactory.from_path('defillama_kg.tsv', create_inverse_triples=False)

In [88]:
training, testing, validation = tf.split([.8, .1, .1])

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [6526, 1486, 1486]


In [75]:
training, testing, validation

(TriplesFactory(num_entities=5422, num_relations=14, create_inverse_triples=True, num_triples=11888, path="D:\hacks\caa\scabi\defillama_kg.tsv"),
 TriplesFactory(num_entities=5422, num_relations=7, create_inverse_triples=False, num_triples=1486, path="D:\hacks\caa\scabi\defillama_kg.tsv"),
 TriplesFactory(num_entities=5422, num_relations=7, create_inverse_triples=False, num_triples=1486, path="D:\hacks\caa\scabi\defillama_kg.tsv"))

In [91]:
# models with literals are not used as it requires literalshape

MODELS = [
#     'autosf', 'boxe', 'compgcn', 'complex', 'conve', 'convkb', 'cooccurrencefiltered',
#     'cp', 'crosse', 'distma', 'distmult', 'ermlp', 'ermlpe', 'fixed', 'hole', 
#     'inductivenodepiece', 'inductivenodepiecegnn', # need inference factory (not yet used)
#     'kg2e', 'mure', 'nodepiece', 'ntn', 'pairre', 'proje', 'quate', 'rescal', 
#      'rgcn', # inverse false
#     'rotate', 'se', 'simple', 'toruse', 'transd', 'transe', 'transf', 'transh', 'transr', 'tucker', 'um'
]
for MODEL in MODELS:
    try:
        DATASET = 'defillama-eth'
        result = pipeline(
            training=training,
            testing=testing,
            validation=validation,
            model=MODEL,
            stopper='early',
            evaluation_relation_whitelist={'has_label'},
            device='cuda:0',
            epochs=500,  # short epochs for testing - you should go higher
            negative_sampler='bernoulli',
            result_tracker='tensorboard',
            training_kwargs=dict(
                checkpoint_name=MODEL+'_'+DATASET+'_checkpoint.pt',
                checkpoint_frequency=50,
            ),
            result_tracker_kwargs=dict(
                experiment_name=MODEL+'_'+DATASET,
            ),
            stopper_kwargs=dict(
                frequency=20,
                patience=5,
                relative_delta=0.001
            ),
        )
    finally:
        pass

In [43]:
result.get_metric('hits@10')

0.30357142857142855

In [11]:
sc_addr = '0x57dD84042ec9507963016596A34EdaD42F7e4CCE'

In [13]:
result.training

TriplesFactory(num_entities=5422, num_relations=7, create_inverse_triples=False, num_triples=11888, path="D:\hacks\caa\scabi\defillama_kg.tsv")

In [44]:
get_tail_prediction_df(result.model, result.training, sc_addr, 'has_label')

,tail_id,tail_label,score,in_training
5421,5421,yield-aggregator,-2.235974,True
4078,4078,options,-6.794979,False
5420,5420,yield,-6.982105,False
1223,1223,dexes,-7.110575,False
879,879,chain,-7.128541,False
...,...,...,...,...
156,156,0x64aa3364F17a4D01c6f1751Fd97C2BD3D7e7f1D5,-10.542206,False
2686,2686,function/renounceoperator,-10.546217,False
19,19,0x0AaCfbeC6a24756c20D41914F2caba817C0d8521,-10.577397,False
2625,2625,function/quack,-10.619660,False
